<a href="https://colab.research.google.com/github/van-dang/FEniCS-Colab/blob/master/PeriodicBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

dolfin version: 2018.1.0


In [40]:
from dolfin import *

class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool(x[0] < DOLFIN_EPS and x[0] > -DOLFIN_EPS and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - 1.0
        y[1] = x[1]
        
mesh = UnitSquareMesh(32, 32)
Ve = FiniteElement("CG", mesh.ufl_cell(), 1)
TH = MixedElement([Ve,Ve])
W = FunctionSpace(mesh, TH, constrained_domain=PeriodicBoundary())

u = Function(W);
u_r, u_i = u[0], u[1]

u0 = Function(W);
u_r0, u_i0 = u0[0], u0[1]


v = TestFunction(W);
v_r, v_i = v[0], v[1]

k = 100;

gnorm = 0.1;

F = 0.1;

g = Expression(("1","1"), degree = 1);

D = 3e-3

um_i = 0.5*(u_i + u_i0)

um_r = 0.5*(u_r + u_r0)

n = FacetNormal(mesh)

def iFT(t, delta, Delta): # integrate ft
    ft1 = t*(t>=0 and t<delta) 
    ft2 = delta*(t>=delta and t<Delta) 
    ft3 = (delta - t + Delta)*(t>=Delta and t<=Delta+delta) 
    return ft1 + ft2 + ft3; 
  
ift = iFT(t, delta, Delta)
r0 = (u_r-u_r0)/k*v_r*dx \
      - gnorm*ift*(dot(g,D*grad(um_i))+dot(grad(um_i),D*g))*v_r*dx \
      + dot(g,D*g)*(gnorm*gnorm*ift*ift)*um_r*v_r*dx \
      + dot(D*grad(um_r),grad(v_r))*dx - dot(D*grad(um_r),n)*v_r*ds
r0 = (u_i-u_i0)/k*v_i*dx \
      + gnorm*ift*(dot(g,D*grad(um_r))+dot(grad(um_r),D*g))*v_i*dx \
      + dot(g,D*g)*(gnorm*gnorm*ift*ift)*um_i*v_i*dx\
      + dot(D*grad(um_i),grad(v_i))*dx - dot(D*grad(um_i),n)*v_i*ds     


NameError: ignored